<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Hands-on Lab: Complete the EDA with SQL</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [18]:
!pip install sqlalchemy==1.4
!pip install ipython-sql
!pip install pandas

  Using cached SQLAlchemy-1.4.0-cp312-cp312-linux_x86_64.whl
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.41
    Uninstalling SQLAlchemy-2.0.41:
      Successfully uninstalled SQLAlchemy-2.0.41
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.0 which is incompatible.
jupyterhub 5.2.1 requires SQLAlchemy>=1.4.1, but you have sqlalchemy 1.4.0 which is incompatible.
  Using cached sqlalchemy-2.0.41-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
Using cached sqlalchemy-2.0.41-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.0
    Uninstalling SQLAlchemy-1.4.0:
      Successfully uninstalled SQLAlchemy-1.4.0


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [19]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [20]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [21]:
%sql sqlite:///my_data1.db

In [22]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

**Note:This below code is added to remove blank rows from table**


In [23]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"** . 

   

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [28]:
from sql import run
import prettytable

# Remove the broken style assignment from config
if hasattr(run, 'PrettyTable'):
    try:
        # Override the ResultSet to avoid using any style
        from sql.run import ResultSet

        def new_init(self, sqlaproxy, config):
            if sqlaproxy is not None:
                self.keys = sqlaproxy.keys()
                self.field_names = self.keys
                list.__init__(self, sqlaproxy.fetchall())
                self.pretty = None  # Disable PrettyTable rendering
            else:
                list.__init__(self, [])

        ResultSet.__init__ = new_init
    except Exception as e:
        print("Override failed:", e)

In [29]:
%sql select distinct "Launch_Site" from SPACEXTABLE

 * sqlite:///my_data1.db


,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'KSC' 


In [30]:
%sql SELECT * FROM SPACEXTABLE WHERE "Launch_Site" LIKE 'KSC%' LIMIT 5;

 * sqlite:///my_data1.db


,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2017-02-19,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
1,2017-03-16,6:00:00,F9 FT B1030,KSC LC-39A,EchoStar 23,5600,GTO,EchoStar,Success,No attempt
2,2017-03-30,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
3,2017-05-01,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
4,2017-05-15,23:21:00,F9 FT B1034,KSC LC-39A,Inmarsat-5 F4,6070,GTO,Inmarsat,Success,No attempt


### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [31]:
%sql SELECT SUM("PAYLOAD_MASS__KG_") AS "Total_Payload" FROM SPACEXTABLE WHERE "Customer" LIKE '%NASA (CRS)%';

 * sqlite:///my_data1.db


,Total_Payload
0,48213


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [32]:
%sql SELECT AVG("PAYLOAD_MASS__KG_") AS "Average_Payload" FROM SPACEXTABLE WHERE "Booster_Version" = "F9 v1.1";

 * sqlite:///my_data1.db


,Average_Payload
0,2928.4


### Task 5

##### List the date where the succesful landing outcome in drone ship was acheived.


_Hint:Use min function_ 


In [33]:
%sql SELECT MIN("Date") AS First_Successful_ASDS_Landing FROM SPACEXTABLE WHERE "Landing_Outcome" LIKE "Success (drone ship)";

 * sqlite:///my_data1.db


,First_Successful_ASDS_Landing
0,2016-04-08


### Task 6

##### List the names of the boosters which have success in ground pad  and have payload mass greater than 4000 but less than 6000


In [34]:
%sql SELECT DISTINCT "Booster_Version" FROM SPACEXTABLE WHERE "Landing_Outcome" = 'Success (ground pad)' AND "PAYLOAD_MASS__KG_" > 4000 AND "PAYLOAD_MASS__KG_" < 6000;

 * sqlite:///my_data1.db


,Booster_Version
0,F9 FT B1032.1
1,F9 B4 B1040.1
2,F9 B4 B1043.1


### Task 7




##### List the total number of successful and failure mission outcomes


In [35]:
%sql SELECT "Mission_Outcome", COUNT(*) AS Total FROM SPACEXTABLE GROUP BY "Mission_Outcome";

 * sqlite:///my_data1.db


,Mission_Outcome,Total
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


### Task 8



##### List all the booster_versions that have carried the maximum payload mass. Use a subquery.


In [36]:
%sql SELECT "Booster_Version", "PAYLOAD_MASS__KG_" FROM SPACEXTABLE WHERE "PAYLOAD_MASS__KG_" = (SELECT MAX("PAYLOAD_MASS__KG_") FROM SPACEXTABLE);



 * sqlite:///my_data1.db


,Booster_Version,PAYLOAD_MASS__KG_
0,F9 B5 B1048.4,15600
1,F9 B5 B1049.4,15600
2,F9 B5 B1051.3,15600
3,F9 B5 B1056.4,15600
4,F9 B5 B1048.5,15600
5,F9 B5 B1051.4,15600
6,F9 B5 B1049.5,15600
7,F9 B5 B1060.2,15600
8,F9 B5 B1058.3,15600
9,F9 B5 B1051.6,15600


### Task 9


##### List the records which will display the month names, succesful landing_outcomes in ground pad ,booster versions, launch_site for the months in year 2017
**Note: SQLLite does not support monthnames. So you need to use substr(Date,6,2) for month, substr(Date,9,2) for date, substr(Date,0,5),='2017' for year.**


In [38]:
%sql SELECT substr(Date, 6, 2) AS Month, "Landing_Outcome", "Booster_Version", "Launch_Site" FROM SPACEXTABLE WHERE substr(Date, 0, 5) = '2017'AND "Landing_Outcome" = 'Success (ground pad)';

 * sqlite:///my_data1.db


,Month,Landing_Outcome,Booster_Version,Launch_Site
0,02,Success (ground pad),F9 FT B1031.1,KSC LC-39A
1,05,Success (ground pad),F9 FT B1032.1,KSC LC-39A
2,06,Success (ground pad),F9 FT B1035.1,KSC LC-39A
3,08,Success (ground pad),F9 B4 B1039.1,KSC LC-39A
4,09,Success (ground pad),F9 B4 B1040.1,KSC LC-39A
5,12,Success (ground pad),F9 FT B1035.2,CCAFS SLC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [39]:
%sql SELECT "Landing_Outcome", COUNT(*) AS "Outcome_Count" FROM SPACEXTABLE WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'GROUP BY "Landing_Outcome" ORDER BY "Outcome_Count" DESC;

 * sqlite:///my_data1.db


,Landing_Outcome,Outcome_Count
0,No attempt,10
1,Success (drone ship),5
2,Failure (drone ship),5
3,Success (ground pad),3
4,Controlled (ocean),3
5,Uncontrolled (ocean),2
6,Failure (parachute),2
7,Precluded (drone ship),1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%203/LAB-String_Patterns_Sorting_Grouping.md.html">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
--!>


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
